In [59]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.random.seed(0)
plt.style.use("ggplot")
import tensorflow as tf
print('Tensorflow version:', tf.__version__)
print('GPU detected:', tf.config.list_physical_devices('GPU'))

# TODO: for the label_data, append a col consist of B if the data have Tag, and I if the data's Tag is NA
# TODO: then perform the lstm

Tensorflow version: 2.10.1
GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [60]:
## read the (cleaned) label data 
label_data = pd.read_table("cleaned_train_data.tsv", delimiter='\t')
num_tag = 0

#special_token_dict = pd.read_tab("multiple_word_tokens.tsv", delimiter='\t')

## read the (cleaned) unlabel data
unlabel_data = pd.read_table("cleaned_data.tsv", delimiter='\t')

In [61]:
# read the labeled data for identifying token
BI_label_data = pd.read_table("partly_cleaned_train_data.tsv", delimiter='\t')

In [62]:
#special_token_dict.head(20)

In [63]:
BI_label_data.head(20)

,Record Number,Title,Token,Tag,C
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell,B
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke,B
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie,B
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN,I
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ,B
5,1,Supreme Nike SB Dunk High By any Means Red US1...,By,Modell,B
6,1,Supreme Nike SB Dunk High By any Means Red US1...,any,NaN,I
7,1,Supreme Nike SB Dunk High By any Means Red US1...,Means,NaN,I
8,1,Supreme Nike SB Dunk High By any Means Red US1...,Red,Farbe,B
9,1,Supreme Nike SB Dunk High By any Means Red US1...,US10,US-Schuhgröße,B


In [64]:
print(unlabel_data.head(20))

    Record Number                                              Title
0               1  Supreme Nike SB Dunk High By any Means Red US1...
1               2  New Balance 530 Männer und Frauen Laufschuhe m...
2               3                     ♥ MICHAEL KORS Sneaker Gr 39 ♥
3               4             New Balance ML 574 EGO Turnschuhe grün
4               5  Nike air jordan 9 og , space jam , hare , bugs...
5               6  NEU SKECHERS Damen Sneakers Turnschuhe Memory ...
6               7                        New Balance 550 White Green
7               8  CAT CATERPILLAR Arise Sneaker Freizeitschuhe T...
8               9  Adidas NMD R1 STLT PK Primeknit Sneaker Turnsc...
9              10  Damen Sneaker Mesh Sportschuhe Laufschuhe Slip...
10             11          Art 505 Schuhe Sneaker Sportschuhe Herren
11             12  C2886 sneaker uomo LANVIN VEGO scarpa nero mul...
12             13  New Balance 574 Herren-Sneaker Low Leder Retro...
13             14  Ecco ST. 1 Lite

In [65]:
## create a dictionary of words and tags
long_string = " ".join(unlabel_data['Title'])
ger_dict = list(set(long_string.split()))
ger_dict.append("ENDPAD")

#tag_dict = list(set(label_data['Tag']))

In [66]:
#print("Unique tag in corpus: ", BI_label_data['Tag'].nunique())
ger_dict[0:20]

['KINDERBERGSCHUHE',
 'LP541',
 '121/2',
 '9baf9',
 '7lpwsnk',
 'Turbon',
 'Trans-Gum',
 'FussballY2E3',
 'Gr.40@look@',
 'B18013t202610602',
 '139900002',
 'GR.M',
 'Pisanie-zmazywanie',
 'AUSGEFALLENENR',
 'Wow!Original',
 'T41632',
 '116926',
 'Vanium',
 '83044cf',
 'Hyperspeed']

In [67]:
#tag_dict[0:20]

In [68]:
class SentenceGetter(object):
    def __init__(self, data):
        self.m_sent = 1
        self.data = data
        # splitting the cols of a sentence into token and tag
        agg_func = lambda s: [(w,t) for w,t in zip(s['Token'].values.tolist(),
                                                  s['C'].values.tolist())]
        self.grouped = self.data.groupby("Record Number").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [81]:
getter = SentenceGetter(BI_label_data)
sentences = getter.sentences
sentences[0:20]

[[('Supreme', 'B'),
  ('Nike', 'B'),
  ('SB', 'B'),
  ('Dunk', 'I'),
  ('High', 'B'),
  ('By', 'B'),
  ('any', 'I'),
  ('Means', 'I'),
  ('Red', 'B'),
  ('US10', 'B'),
  ('EU44', 'B'),
  ('Supreme', 'B'),
  ('Box', 'B'),
  ('Logo', 'B'),
  ('Air', 'B'),
  ('Force', 'I')],
 [('New', 'B'),
  ('Balance', 'I'),
  ('530', 'B'),
  ('Männer', 'B'),
  ('und', 'B'),
  ('Frauen', 'B'),
  ('Laufschuhe', 'B'),
  ('mit', 'B'),
  ('Buchstaben', 'B'),
  ('N', 'I'),
  ('bequeme', 'B'),
  ('Laufschuhe', 'B')],
 [('♥', 'B'),
  ('MICHAEL', 'B'),
  ('KORS', 'I'),
  ('Sneaker', 'B'),
  ('Gr', 'B'),
  ('39', 'B'),
  ('♥', 'B')],
 [('New', 'B'),
  ('Balance', 'I'),
  ('ML', 'B'),
  ('574', 'I'),
  ('EGO', 'I'),
  ('Turnschuhe', 'B'),
  ('grün', 'B')],
 [('Nike', 'B'),
  ('air', 'B'),
  ('jordan', 'I'),
  ('9', 'B'),
  ('og', 'I'),
  (',', 'B'),
  ('space', 'B'),
  ('jam', 'B'),
  (',', 'B'),
  ('hare', 'B'),
  (',', 'B'),
  ('bugs', 'B'),
  ('bunny', 'I'),
  (',', 'B'),
  ('EU', 'B'),
  ('45', 'I'),
  ('/', 

In [78]:
ger_label_dict = list(set(label_data['Token']))
#train2idx = {w: i+1 for i, w in enumerate(ger_label_dict)}
#tag2idx = {w: i for i, w in enumerate(tag_dict)}
ger2idx = {w: i+1 for i, w in enumerate(ger_dict)}
BI2idx = {"B":0, "I": 1}

In [71]:
ger2idx

{'KINDERBERGSCHUHE': 1,
 'LP541': 2,
 '121/2': 3,
 '9baf9': 4,
 '7lpwsnk': 5,
 'Turbon': 6,
 'Trans-Gum': 7,
 'FussballY2E3': 8,
 'Gr.40@look@': 9,
 'B18013t202610602': 10,
 '139900002': 11,
 'GR.M': 12,
 'Pisanie-zmazywanie': 13,
 'AUSGEFALLENENR': 14,
 'Wow!Original': 15,
 'T41632': 16,
 '116926': 17,
 'Vanium': 18,
 '83044cf': 19,
 'Hyperspeed': 20,
 'Komfo': 21,
 'CM23': 22,
 'US4-Größe': 23,
 'PULCHRUM': 24,
 '4d9f96a': 25,
 '826595': 26,
 'Kimpu': 27,
 '17a0b17': 28,
 '5531a0f': 29,
 'BR668-42': 30,
 'Ivory-Gum': 31,
 '381a37': 32,
 '45264': 33,
 '0a5b04': 34,
 'touchsala': 35,
 'superweichen': 36,
 '757483': 37,
 '101.177705': 38,
 '689bd4c': 39,
 '9fdabe': 40,
 '988f34': 41,
 'S25248-5': 42,
 'KaroTOP': 43,
 '2790-SUEW': 44,
 '6bd8': 45,
 '621709': 46,
 '1fb767e': 47,
 '111299': 48,
 '5740VD': 49,
 '5001-028_35': 50,
 '84a0': 51,
 '631461': 52,
 'B7733': 53,
 'J03058': 54,
 '7f0a96b': 55,
 '831069-600': 56,
 '848187200': 57,
 'i158': 58,
 'Razzoni': 59,
 'NEU+NEU+NEU': 60,
 '17

In [72]:
#tag2idx

In [76]:
## plot the length of each titles, commented out for debugging efficiency

# TODO: đếm số token trong câu
# plt.hist(BI_label_data["Title"].str.len(), bins = 20)

In [79]:
#TODO: fix this

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = 98
X = [[ger2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding='post', value= len(ger_dict)-1)

y = [[BI2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding='post', value=tag2idx["No Tag"])
y = [to_categorical(i, num_classes=len(tag_dict)) for i in y]

KeyError: '/\\"\tNo Tag"'

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)

In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

In [ ]:
## creating a bi-directional RNN
input_word = Input(shape=(max_len, ))
model = Embedding(input_dim=len(ger_dict), output_dim=max_len, input_length=max_len)(input_word)
model = SpatialDropout1D(.1)(model)
model = Bidirectional(LSTM(units=256, return_sequences=True, recurrent_dropout=.1))(model)
out = TimeDistributed(Dense(len(tag_dict), activation = 'softmax'))(model)
model = Model(input_word, out)
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss = 'categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from livelossplot.tf_keras import PlotLossesCallback

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, verbose=0, mode='max', restore_best_weights=False)
callbacks = [PlotLossesCallback(), early_stopping]

history = model.fit(
    x_train, np.array(y_train),
    validation_split=.2,
    batch_size=64,
    epochs=10,
    verbose=1,
    callbacks=callbacks
)

In [20]:
model.evaluate(x_test, np.array(y_test))

16/16 [==============================] - 0s 14ms/step - loss: 2.6785 - accuracy: 0.3524


[2.678514242172241, 0.352400004863739]

In [21]:
predict = []

for i in range(len(unlabel_data['Title'])):
    word_indices = []
    for word in unlabel_data['Title'][i].split():
        word_indices.append(ger2idx[word])
    predict.append(word_indices)

predict = pad_sequences(maxlen=max_len, sequences=predict, padding='post', value= len(ger_dict)-1) 

In [34]:
import csv
with open('records.tsv', 'w', newline='', encoding = "utf-8") as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    #len(unlabel_data['Title'])
    for i in range(10):
        p = model.predict(np.array([predict[i]]))
        p = np.argmax(p, axis=-1)
        list_length = len(unlabel_data['Title'][i].split())
        for tok, pred in zip(predict[i][0:list_length], p[0][0:list_length]):
            writer.writerow([i + 1, ger_dict[tok - 1], tag_dict[pred]])
        

1/1 [==============================] - 0s 32ms/step


In [31]:


print("{:20}\t {}\n".format("Token", "Predict tag"))
print("-"*40)
list_length = len(unlabel_data['Title'][10].split())
for tok, pred in zip(predict[10][0:list_length], p[0][0:list_length]):
    print("{:20}\t{}".format(ger_dict[tok-1], tag_dict[pred]))

Token               	 Predict tag

----------------------------------------
Art                 	No Tag
505                 	No Tag
Schuhe              	No Tag
Sneaker             	No Tag
Sportschuhe         	No Tag
Herren              	No Tag
6
